# 개선해야하는 부분

    1) 유사도 0.6 이상 조건 넣기
    2) [사진,포토] 등 불필요한 단어 제거
    3) 불용어처리 가능하게 하기 

# 제목유사도 측정

In [1]:
import re
import pandas as pd
import numpy as np
import json
#from bs4 import Beautiful Soup
# from nltk.corpus import Beautiful Soup
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import konlp
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#헤더추가 (title,content,time)
all =pd.read_csv('dataset3000.csv',names=['title','content','time','nan'])
all.head()
all.shape
all.dtypes

title       object
content     object
time        object
nan        float64
dtype: object

In [3]:
all.head()

,title,content,time,nan
0,"'[사진]조여정,'우아한 플라워원피스 입고''",'제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...,2019-07-20 16:40,NaN
1,'[프로필]국세청 과장급 전보자(7월 22일 자)','조사기획1계장 ▲경주세무서장 ▲국세청 인도네시아 주재관 ▲외교부 파견 ▲서울지방국...,2019-07-20 16:40,NaN
2,'[조이HD]춘사영화제 참석한 봉준호 감독-한진원 작가','말레이시아 Finas(말레시아 영화진흥위)의 신임 대표인 Ahmad Idham을 ...,2019-07-20 16:40,NaN
3,"'[사진]클라라,'시선 올킬 블루 롱드레스''",'제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...,2019-07-20 16:40,NaN
4,"'[사진]7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''",'제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...,2019-07-20 16:40,NaN


# trian 과 test 나누기 

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
train, test = train_test_split(all, test_size=0.2)

In [6]:
train.info

<bound method DataFrame.info of                                                   title  \
2863                   '“파월이 또 우릴 실망시켰다”‥트럼프 맹비난(종합2보)'   
2440                   ''We Boom' 영국 음악 전문 매거진 NME도...'   
668                         '이민정 ‘계절 앞서가는 공항패션’[포토엔HD]'   
1604                        '中, 포스코 등 스테인리스에 반덤핑 관세 부과'   
2786                             '올해 PIS는 '친환경 섬유'가 대세'   
2259                  '배드민턴 김소영-공희용, 일본 꺾고 일본오픈 우승(종합)'   
1704                            '인도양 바다 보며 체험하는 플라잉 요가'   
1754  '국내 최대 스타트업 페어 '넥스트라이즈 서울' 개막…\한국판 CES·MWC 만들겠...   
32                                   '[사진]정애연,'단아한 매력''   
959         '美빌보드 “여자친구, 꾸준한 성장” 월드 앨범 차트 톱10 진입 집중 조명'   
2818                  ''팔아야귀국' 나르샤는 신봉선에게 우리 이제 1시간...'   
314   ''女개인혼영 200m 결선' 김서영, \다른 사람 아닌 나를 위해 후회없이 도전\...   
360               '인도네시아어, 2019년 07월 11일 오늘의 외국어 한마디는?'   
1969               '‘넥스트라이즈 2019’ 성료..글로벌 스타트업에 8천명 관심'   
2376               '여행지 맛집 그대로…유통·식품업계, 해외 인기 메뉴에 ‘주목’'   
1079                    

In [7]:
test.info

<bound method DataFrame.info of                                                   title  \
1521                '韓 화물선 해적에 피습·2명 부상…“해상 특수경비원 없었다”'   
2164                           '더 레지던스 빈탄이 자랑하는 릴랙스 공간'   
1897          '보잉, 2분기 사상 최대 손실…\737맥스 생산 중단 검토할 상황\"'"   
1169                                '시노펙스, 印尼 수처리사업 협약'   
1789               '크래프톤-아크시스템웍스, 미스트오버 아시아 지역 퍼블리싱 계약'   
1165             '우크라이나 국회, 아동 성범죄자 ‘화학적 거세 의무화’ 법안 가결'   
328   '전북 군산 어청도 해역서 규모 2.0 지진 발생…7월 들어 6번째 지진 발생으로...'   
2614                   '경북, 中企 수출 돕는 청년무역 전문인력 104명 탄생'   
2527                   '경북청년무역사관학교, 청년 무역인력 104명 새로 배출'   
2355               '에이스, K-POP 가수 중 유일하게 자카르타 푸드 페어 참석'   
1022                      '발리 지진 발생…규모 5.7, 쓰나미 경보는 없어'   
943                               '빈탄 골프장에서 즐기는 스페셜 디너'   
1058          '웨이션브이 루카스, 글로벌 광고계 블루칩으로 급부상‥실시간 트렌드 장악'   
819                                     '새로운 색깔 입은 리모와'   
1810                  '신라대-인니 한국기업, 교육과정 운영…취업 활성화 꾀한다'   
1281              '블락비 유

In [8]:
train_title=list(train['title'])

In [9]:
for i in range(len(train_title)) :
    train_title[i] = re.sub("[^가-힣ㄱ-ㅎ ㅏ-ㅣ\\s]","", train_title[i])

print(train_title)

['파월이 또 우릴 실망시켰다트럼프 맹비난종합보', '  영국 음악 전문 매거진 도', '이민정 계절 앞서가는 공항패션포토엔', ' 포스코 등 스테인리스에 반덤핑 관세 부과', '올해 는 친환경 섬유가 대세', '배드민턴 김소영공희용 일본 꺾고 일본오픈 우승종합', '인도양 바다 보며 체험하는 플라잉 요가', '국내 최대 스타트업 페어 넥스트라이즈 서울 개막한국판  만들겠', '사진정애연단아한 매력', '빌보드 여자친구 꾸준한 성장 월드 앨범 차트 톱 진입 집중 조명', '팔아야귀국 나르샤는 신봉선에게 우리 이제 시간', '개인혼영  결선 김서영 다른 사람 아닌 나를 위해 후회없이 도전 ', '인도네시아어 년 월 일 오늘의 외국어 한마디는', '넥스트라이즈  성료글로벌 스타트업에 천명 관심', '여행지 맛집 그대로유통식품업계 해외 인기 메뉴에 주목', '클럽메드 빈탄 아일랜드 공중그네 클래스', '글로벌 워치 한달새 세 번이나 몸 떤 메르켈 총리건강이상설', '껍질이 더 좋아치킨집 매출 배 늘려 준 신제품은', '김야니 일 인스타그램을 통해 일상 공개', '인니 몰루카제도 규모  지진쓰나미 경보 없어', '포토 인니 강진 여진 공포에 집 밖으로', '화물선 씨케이블루벨호 싱가포르서 무장 해적에 피습무기 휴대한 특수경', '사자 판타지아 국제영화제 초청해외 개국 동시기 개봉', '사진상체 노출 클라라이보다 더 아찔할 수 없어', '서울대 이병천 교수 연구비 비리 의혹 감사 착수', '스타일이 이끄는 패션코드   참가사 모집', '배드민턴 복식 김소영공희용 일본오픈 우승', '해외반응사자 판타지아영화제 초청개국 선판매공식', '글로벌 진출 원년 선언커가는 종근당의 자신감', '더셀럽포토수영 예쁘게 찍고 올게요', '종근당\xa0인도네시아 항암제 공장 준공글로벌 진출 가속화', '글로벌 쇼핑 플랫폼  동남아 온라인 시장 진출 교두보', '팔아야귀국 시간이 흘러도 손님들은 줄지 않았다', '포토 인도네시아 국세청장회의', '정의선 현대차그룹 수석부회장 조코 위도도 인

# train title 문장의 명사만 추가한 리스트 만들기

### 여기선 TF-idf 사용

참고사이트: https://code-ing.tistory.com/2?category=728528

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

okt=Okt()

doc_nouns_list=[]

for doc in train_title:
    nouns=okt.nouns(doc)
    doc_nouns=''
    
    for noun in nouns:
        doc_nouns+=noun + ' '
    doc_nouns_list.append(doc_nouns)
    
for i in range(0,len(train_title)):
    print('doc' +str(i+1)+ ' : ' +str(doc_nouns_list[i]))

C:\Users\glass\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


doc1 : 파월 또 우릴 실망 트럼프 맹 비난 종합 보 
doc2 : 영국 음악 전문 도 
doc3 : 이민정 계절 공항 패션 포토 
doc4 : 포스코 등 스테인리스 반덤핑 관세 부과 
doc5 : 올해 친환경 섬유 대세 
doc6 : 배드민턴 김소영 공 일본 일본 오픈 우승 종합 
doc7 : 인도양 바다 체험 플라잉 요가 
doc8 : 국내 최대 스타트업 페어 넥스트 라이즈 서울 개막 한국판 
doc9 : 사진 정애연 단아 매력 
doc10 : 빌보드 여자친구 성장 월드 앨범 차트 톱 진입 집중 조명 
doc11 : 아야 귀국 나르샤 신봉선 우리 이제 시간 
doc12 : 개인혼영 결선 김서영 다른 사람 위해 후회 도전 
doc13 : 인도네시아어 년 월 일 오늘 외국어 한마디 
doc14 : 넥스트 라이즈 료 글로벌 스타트업 천명 관심 
doc15 : 여행지 맛집 그대로 유통 식품 업계 해외 인기 메뉴 주목 
doc16 : 클럽메드 빈탄 아일랜드 공중그네 클래스 
doc17 : 글로벌 워치 달새 세 번 몸 메르켈 총리 건강 이상설 
doc18 : 껍질 더 치킨 집 매출 배 준 신제품 
doc19 : 일 인스타그램 통해 일상 공개 
doc20 : 인니 루카 제도 규모 지진 쓰나미 경보 
doc21 : 포토 인니 강진 여진 공포 집 밖 
doc22 : 화물선 씨 케이 블루 호 싱가포르 무장 해적 피습 무기 휴대 특수 경 
doc23 : 사자 판타지아 국제 영화제 초청 해외 개국 시기 개봉 
doc24 : 사진 상체 노출 클라라 더 수 
doc25 : 서울대 이병천 교수 구비 비리 의혹 감사 착수 
doc26 : 스타일 패션 코드 참가 사 모집 
doc27 : 배드민턴 복식 김소영 공 일본 오픈 우승 
doc28 : 해외반응 사자 판타지아 영화제 초청 개국 선 판매 공식 
doc29 : 글로벌 진출 원년 선언 종근당 자신감 
doc30 : 더 셀럽 포토 수영 
doc31 : 종근당 인도네시아 항암제 공장 준공 글로벌 진출 가속 
doc32 : 글로벌 쇼핑 

In [11]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

document_distances=(tfidf_matrix*tfidf_matrix.T)

print('유사도 분석을 위한' + str(document_distances.get_shape()[0])+'x'+str(document_distances.get_shape()[1])+'matrix를 만들었습니다')
print(document_distances.toarray())     

유사도 분석을 위한2298x2298matrix를 만들었습니다
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [12]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

document_distances=(tfidf_matrix*tfidf_matrix.T)

print()
print(document_distances.toarray())


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [13]:
# 유사도 0.6 이상 조건 넣기 

# 중복줄이기 

In [14]:
distanceArray = document_distances.toarray()
check = [True for _ in range(len(distanceArray))]

In [15]:
for i in range(len(distanceArray)):
    if(check[i] == False):
        continue
    for j in range(len(distanceArray)):
        if(i == j):
            continue
        else :
            if(check[j] == True and distanceArray[i][j] >= 0.6):
                check[j] = False

In [16]:
filterList = []
print(len(check))
for i in range(len(check)):
    if(check[i] == True):
        filterList.append(train_title[i])
print(len(filterList))

2298
1403


In [17]:
print(filterList)

['파월이 또 우릴 실망시켰다트럼프 맹비난종합보', '  영국 음악 전문 매거진 도', '이민정 계절 앞서가는 공항패션포토엔', ' 포스코 등 스테인리스에 반덤핑 관세 부과', '올해 는 친환경 섬유가 대세', '배드민턴 김소영공희용 일본 꺾고 일본오픈 우승종합', '인도양 바다 보며 체험하는 플라잉 요가', '국내 최대 스타트업 페어 넥스트라이즈 서울 개막한국판  만들겠', '사진정애연단아한 매력', '빌보드 여자친구 꾸준한 성장 월드 앨범 차트 톱 진입 집중 조명', '팔아야귀국 나르샤는 신봉선에게 우리 이제 시간', '개인혼영  결선 김서영 다른 사람 아닌 나를 위해 후회없이 도전 ', '인도네시아어 년 월 일 오늘의 외국어 한마디는', '넥스트라이즈  성료글로벌 스타트업에 천명 관심', '여행지 맛집 그대로유통식품업계 해외 인기 메뉴에 주목', '클럽메드 빈탄 아일랜드 공중그네 클래스', '글로벌 워치 한달새 세 번이나 몸 떤 메르켈 총리건강이상설', '껍질이 더 좋아치킨집 매출 배 늘려 준 신제품은', '김야니 일 인스타그램을 통해 일상 공개', '인니 몰루카제도 규모  지진쓰나미 경보 없어', '포토 인니 강진 여진 공포에 집 밖으로', '화물선 씨케이블루벨호 싱가포르서 무장 해적에 피습무기 휴대한 특수경', '사자 판타지아 국제영화제 초청해외 개국 동시기 개봉', '사진상체 노출 클라라이보다 더 아찔할 수 없어', '서울대 이병천 교수 연구비 비리 의혹 감사 착수', '스타일이 이끄는 패션코드   참가사 모집', '해외반응사자 판타지아영화제 초청개국 선판매공식', '글로벌 진출 원년 선언커가는 종근당의 자신감', '더셀럽포토수영 예쁘게 찍고 올게요', '종근당\xa0인도네시아 항암제 공장 준공글로벌 진출 가속화', '글로벌 쇼핑 플랫폼  동남아 온라인 시장 진출 교두보', '팔아야귀국 시간이 흘러도 손님들은 줄지 않았다', '포토 인도네시아 국세청장회의', '정의선 현대차그룹 수석부회장 조코 위도도 인니 대통령과 면담', '광주 세계인권도시포럼 세

In [ ]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

document_distances=(tfidf_matrix*tfidf_matrix.T)

print('유사도 분석을 위한' + str(document_distances.get_shape()[0])+'x'+str(document_distances.get_shape()[1])+'matrix를 만들었습니다')
print(document_distances.toarray())    

In [22]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrixA = tfidf_vectorizer.fit_transform(doc_nouns_list[:20])

In [24]:
doc_nouns_list[:10]

['파월 또 우릴 실망 트럼프 맹 비난 종합 보 ',
 '영국 음악 전문 도 ',
 '이민정 계절 공항 패션 포토 ',
 '포스코 등 스테인리스 반덤핑 관세 부과 ',
 '올해 친환경 섬유 대세 ',
 '배드민턴 김소영 공 일본 일본 오픈 우승 종합 ',
 '인도양 바다 체험 플라잉 요가 ',
 '국내 최대 스타트업 페어 넥스트 라이즈 서울 개막 한국판 ',
 '사진 정애연 단아 매력 ',
 '빌보드 여자친구 성장 월드 앨범 차트 톱 진입 집중 조명 ']

In [23]:
tfidf_matrixA

<10x55 sparse matrix of type '<class 'numpy.float64'>'
	with 56 stored elements in Compressed Sparse Row format>